In [1]:
import random
import json
import  pickle
import numpy as np

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
from tensorflow.keras.models import load_model

In [4]:
lemmatizer=WordNetLemmatizer()

In [5]:
intents=json.loads(open('intent.json',encoding='utf-8').read())

In [6]:
words=pickle.load(open('words.pkl','rb'))

In [7]:
classes=pickle.load(open('classes.pkl','rb'))

In [8]:
model=load_model('chatbot_model.h5')

In [9]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [10]:
def bag_of_words(sentence,words,show_details=True):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
                if show_details:
                    print("found in bag:%w"% word)
    return np.array(bag)

In [11]:
def predict_class(sentence,model):
    bow=bag_of_words(sentence,words,show_details=False)
    res=model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD=0.25
    results=[[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list=[]
    for r in results:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list

In [ ]:
def get_response(intents_list,intent_json):
    tag=intents_list[0]['intent']
    list_of_intents=intent_json['intents']
    for i in list_of_intents:
        if(i['tag']==tag):
            result= random.choice(i['responses'])
            break
        else:
            result=" Sorry,I do not understand.Please enter a number from the list"
    return result

print("Hi! I am your chatbuddy from Polaris bank")

while True:
    message=input("")
    ints=predict_class(message,model)
    resp=get_response(ints,intents)
    print(resp)

Hi! I am your chatbuddy from Polaris bank
Good day
1/1 [==============================] - 2s 2s/step
Hi, I'm Rob.I can help you with the right information needed to resolve the following issues. Please enter/type a number from the list below to begin with:                                                                                                         1. Opening an account                                                                                                                       2. Failed transaction                                                                                                                       3. Loan application                                                                                                                         4. How to check account balance                                                                                                             5. How to make a transfer                                                      